In [ ]:
import pandas as pd
import numpy as np
import csv as csv
import os
import sys
import codecs
import time
import datetime

import seaborn as sns 
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from matplotlib.font_manager import FontProperties
from collections import Counter

from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
import plotly.plotly as py
import plotly.graph_objs as go
import psutil
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
myfont = FontProperties(fname=r'/var/anaconda3/jupyter/derrick.wu/msjh.ttc')
my_font = matplotlib.font_manager.FontProperties(fname=r'/var/anaconda3/jupyter/derrick.wu/msjh.ttc')
pd.set_option('display.max_columns',1000) 

In [1]:
! python -m pip install -- upgrade pip

  Could not fetch URL https://pypi.org/simple/upgrade/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/upgrade/ (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)) - skipping


  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/upgrade/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/upgrade/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/upgrade/
  Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/upgrade/
  Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connectio

In [ ]:
Data=pd.read_csv(r'C:\Users\derrick.wu\Documents\DW\Resign_Rate_Model.csv',header=0,encoding = 'utf-8',low_memory=False)

In [2]:
! pip install catboost

  Could not fetch URL https://pypi.org/simple/catboost/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/catboost/ (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)) - skipping


  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/catboost/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/catboost/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/catboost/
  Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:833)'),)': /simple/catboost/
  Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after conne

In [ ]:
#讀入uc01
t2=time.time()
uc01 = pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/EUC_files/Policy_UC01_201906.csv',header=0,encoding = 'utf-8',low_memory=False)
print("Reading uc01.CSV costs "+str(time.time()-t2)+" seconds")
#讀uc02
t2=time.time()
uc02 = pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/EUC_files/UC02_201906.csv',header=0,encoding = 'utf-8',error_bad_lines=False,low_memory=False)
print("Reading uc02.CSV costs "+str(time.time()-t2)+" seconds")

#讀uc05
t2=time.time()
uc05 = pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/EUC_files/CusBase_UC05_201906.csv',header=0,encoding = 'utf-8',low_memory=False)
print("Reading uc05.CSV costs "+str(time.time()-t2)+" seconds")

#讀aa01
t2=time.time()
aa01= pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/EUC_files/Agent_base_AA01_201906.csv',header=0,encoding = 'utf-8',error_bad_lines=False,low_memory=False)
print("Reading aa01.CSV costs "+str(time.time()-t2)+" seconds")
                   
#讀aa02
t2=time.time()
aa02= pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/EUC_files/Office_base_AA02_201906.csv',header=0,encoding = 'utf-8',error_bad_lines=False,low_memory=False)
print("Reading aa02.CSV costs "+str(time.time()-t2)+" seconds")                   
#讀income data
income_data=pd.read_csv('/var/anaconda3/jupyter/derrick.wu/income_data.csv',header=0,encoding = 'utf-8',error_bad_lines=False,dtype={'業務員代碼':str,'AGNTNUM':str,'工作月份':str},low_memory=False)
income_data=income_data.drop('Unnamed: 0',axis=1)
income_data['工作月份']=np.array(income_data.工作月份).astype('<U6').tolist()

In [ ]:
income_data['工作月份']=income_data['工作月份'].astype(int)

In [ ]:
income_data['AGNTNUM']=income_data['AGNTNUM'].astype(int)

In [ ]:
                   
#讀subsection
t2=time.time()
section_table_replace = pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/MappingTables/subsection對應十分類_例外抽換表.csv',header=0,encoding = 'utf-8',error_bad_lines=False)
print("Reading Section_table.CSV costs "+str(time.time()-t2)+" seconds")
t2=time.time()
section_table = pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/MappingTables/subsection對應十分類.csv',header=0,encoding = 'utf-8',error_bad_lines=False)
print("Reading Section_table.CSV costs "+str(time.time()-t2)+" seconds")

#合併UC01UC02
uc0102=uc01.merge(uc02, how='left', on=None, left_on=['Sysfrom','保單號碼'], right_on=['Sysfrom','保單號碼'], left_index=False, right_index=False, sort=False)
#uc0102_AG = uc0102[uc0102['業務來源'].isin(['AG'])]
#uc0102_AG_sub=uc0102_AG.merge(section_table, how='left', on=None, left_on=['Sub-Section'], right_on=['subsection'], left_index=False, right_index=False, sort=False)

#讀入業務資料
#Agent_Data=pd.read_csv('/var/anaconda3/jupyter/share/002_SourceData/EUC_files/Agent_base_AA01_201906.csv',header=0,encoding = 'utf-8',error_bad_lines=False)
#合併業務資料
#uc0102_AG_Agent=uc0102_AG_sub_noNA.merge(Agent_Data, how='left', on=None, left_on=['招攬業代id'], right_on=['業務員ID'], left_index=False, right_index=False, sort=False)


In [ ]:
uc0102

In [ ]:
uc01.head()

In [ ]:
aa01.head(1)

In [ ]:
uc05.head(1)

In [ ]:
aa02.head()

In [ ]:
aa01.head()

In [ ]:
aa0102=aa01.merge(aa02, how='left', on=None, left_on=['通訊處代碼'], right_on=['通訊處代碼'], left_index=False, right_index=False, sort=False)


In [ ]:
#保留身分別=2的
aa0102= aa0102[(aa0102['身分別']==2)]
#篩掉無效資料
aa_data = pd.concat([aa0102[['業務員代碼','業務員姓名','業務員ID','生日','本次職等變更日','最近舉績日','原始報聘日期','解聘日期 ','區部','職務代號']]], axis=1)
#aa_data=aa_data[aa_data['生日']<20010724]
#temp = aa_data.copy()


In [ ]:
aa_data=aa_data.reset_index(drop=1)

In [ ]:
aa_data.shape

In [ ]:
#time形式=201301
def _10mark(data,time,period):
    time=time*100
    time_temp = data[(data['原始報聘日期']<=time)&(data['解聘日期 ']>=time)]
    if int(time/100)%100+period>12:
        year=int(time/10000)+1
        month = (time%100+period)%12
        timeplus=year*10000+month*100
    else:
        timeplus=time+(period)*100
    _1=time_temp[(time_temp['解聘日期 ']<=timeplus)]
    _2=time_temp[~(time_temp['解聘日期 ']<=timeplus)]
    _1['Left_3m']=1
    _2['Left_3m']=0
    time_temp=_1.append(_2)
    time_temp=time_temp.reset_index(drop=1)
    time_temp['Time']=int(time/100)
    return(time_temp)

In [ ]:
Final.業務員代碼[1]

In [ ]:
time_temp.append(time_temp,ignore_index=1).業務員代碼[0]

In [ ]:
Final=pd.DataFrame()
#判斷取往後'period'個月是否離職
period=3
#如起始為20160101則20160401之前離職的會被標記為1

for i in [2013,2014,2015,2016,2017]:
    for j in range(1,13):
        time=i*100+j
        Final=Final.append(_10mark(aa_data,time,period),ignore_index=1)

In [ ]:
Data_incomeandleft=Final.merge(income_data,how='left',left_on=['業務員代碼','Time'],right_on=['AGNTNUM','工作月份'],suffixes=('', '_y'))

In [ ]:
type (income_data.AGNTNUM[0])

In [ ]:
Data_incomeandleft.dropna(how='any',thresh=None, subset=['FYC'], inplace=True)

In [ ]:
Data_incomeandleft.reset_index(drop=1)

In [ ]:
Data=Data_incomeandleft.copy()

In [ ]:
Data_incomeandleft

In [ ]:
Data.columns

In [ ]:
#篩掉收入為零的資料
Data=Data[(Data['Total']>0)]

In [ ]:
#加入年齡、年資
Data['年齡']=round((Data['Time']*100-Data['生日'])/10000)
Data['年資']=round((Data['Time']*100-Data['原始報聘日期'])/10000)
#加入性別
a=np.array(Data['業務員ID'].tolist()).astype('<U2')
Gender_list_=[]
for i in a:
    Gender_list_=Gender_list_+[i[-1]]
Data['性別']=list_

In [ ]:
#薪資成長率
Agent_List=Data.業務員ID.unique()
Wage_Sector=Data.groupby(by='業務員ID')
c=0
t1=time.time()
for agent in Agent_List:
    c=c+1
    one_agent_table=Wage_Sector.get_group(agent).reset_index(drop=1)
    yoy_list=[]
    income_list=one_agent_table['Total']
    for index in range(len(income_list)):
        if index ==0:
            yoy_list.append(0)
        else :
            yoy_list.append((income_list[index]-income_list[index-1])/income_list[index-1])
    one_agent_table['Wage_YoY']=yoy_list
    if c==1:
        df=one_agent_table
    else:
        df=df.append(one_agent_table,ignore_index=True)
    if c%1000==0:
        t2=time.time()
        print(str(int(t2-t1))+' seconds used and '+str(round((c/15976),2))+' is finished')
    

In [ ]:
#by年資的薪資水平
#先group by職級
for exp_table in Data.groupby('exp'):
    exp_table=exp_table[1]
    for wage_rk_table in exp_table.groupby('work_month'):
        wage_rk_table=wage_rk_table[1]
        temp_list=[stats.percentileofscore(wage_rk_table['Total'], a, 'rank') for a in wage_rk_table['Total']]
        wage_rk_table['W_Rank_by_exp']=temp_list
        for index in wage_rk_table.index:
            Data.at[index,'W_Rank_by_exp']=wage_rk_table.at[index,'W_Rank_by_exp']

In [ ]:
#by職級的薪資水平
#先group by職級
for pos_table in Data.groupby('Pos_Ini'):
    pos_table=pos_table[1]
    for wage_rk_table in pos_table.groupby('work_month'):
        wage_rk_table=wage_rk_table[1]
        temp_list=[stats.percentileofscore(wage_rk_table['Total'], a, 'rank') for a in wage_rk_table['Total']]
        wage_rk_table['W_Rank_by_Pos']=temp_list
        for index in wage_rk_table.index:
            Data.at[index,'W_Rank_by_Pos']=wage_rk_table.at[index,'W_Rank_by_Pos']

In [ ]:
#by職級的薪資水平的變動率(與前一個月比較)
for ID_table in  Data.groupby('業務員ID'):
    ID_table=ID_table[1].sort_values(by='work_month')
    a=0
    mark_list=[]
    for rank in ID_table['W_Rank_by_Pos']:
        a=a+1
        if a==1:
            temp=rank
            mark_list.append(0)
        elif rank>=temp:
            temp=rank
            mark_list.append(1)
        elif rank<temp:
            temp=rank
            mark_list.append(-1)
        else:
            print('aaaa')
            temp=rank
            mark_list.append(2)
    ID_table['delta_1m']=mark_list
    for index in ID_table.index:
        Data.at[index,'delta_1m']=ID_table.at[index,'delta_1m']

In [ ]:
#by職級的薪資水平的變動率(與前兩個月比較)
for ID_table in  Data.groupby('業務員ID'):
    ID_table=ID_table[1].sort_values(by='work_month')
    a=0
    mark_list=[]
    for rank in ID_table['W_Rank_by_Pos']:
        a=a+1
        if a==1:
            temp_small=rank
            mark_list.append(0)
        elif a==2:
            temp_small_2=rank
            mark_list.append(0)
        elif a>2:
            if rank>=temp_small:
                mark_list.append(1)
                temp_small=temp_small_2
                temp_small_2=rank
            elif rank<temp_small:
                mark_list.append(-1)
                temp_small=temp_small_2
                temp_small_2=rank
            else:
                mark_list.append(0)
            
            
    ID_table['delta_2m']=mark_list
    for index in ID_table.index:
        Data.at[index,'delta_2m']=ID_table.at[index,'delta_2m']

In [ ]:
#by職級的薪資水平的變動率(與前三個月比較)
for ID_table in  Data.groupby('業務員ID'):
    ID_table=ID_table[1].sort_values(by='work_month')
    a=0
    mark_list=[]
    for rank in ID_table['W_Rank_by_Pos']:
        a=a+1
        if a==1:
            temp_small=rank
            mark_list.append(0)
        elif a==2:
            temp_small_2=rank
            mark_list.append(0)
        elif a==3:
            temp_small_3=rank
            mark_list.append(0)
        elif a>3:
            if rank>=temp_small:
                mark_list.append(1)
                temp_small=temp_small_2
                temp_small_2=temp_small_3
                temp_small_3=rank
            elif rank<temp_small:
                mark_list.append(-1)
                temp_small=temp_small_2
                temp_small_2=temp_small_3
                temp_small_3=rank
            else:
                mark_list.append(0)
            
            
    ID_table['delta_3m']=mark_list
    for index in ID_table.index:
        Data.at[index,'delta_3m']=ID_table.at[index,'delta_3m']

In [ ]:
mark_list

In [ ]:
Data.sort_values(by=)

In [ ]:
Data[Data['業務員ID']=='A101269013']

In [ ]:
df.to_csv('業務脫退率大表_0813.csv',encoding='utf-8')

In [ ]:
df['職務代號_開頭']=np.array(df['職務代號'].tolist()).astype('<U1')

In [ ]:
s=df.groupby(by='職務代號_開頭').size()
list_position=df.職務代號_開頭.unique()
for i in list_position:
    table=s.get_group(i).reset_index(drop=True)
    

In [ ]:
a=df['年齡']
quantile_list[]

In [ ]:
def seniority_rank(row):
    a = map_rank[row['年齡']]
    if row['本次報聘年資']<0:
        return 0
    elif row['本次報聘年資']<quantile_list[a][0]:
        return 1
    elif row['本次報聘年資']<quantile_list[a][1]:
        return 2
    elif row['本次報聘年資']<quantile_list[a][2]:
        return 3
    else:
        return 4
df['職位年齡比']=df.apply(lambda row: seniority_rank(row), axis=1)

In [ ]:
s=Data.groupby('性別')

In [ ]:
Data=Data[Data['性別'].isin(['1','2'])]

In [ ]:
Data=Data.reset_index(drop=1)

In [ ]:
Data.to_csv('業務脫退率大表.csv',encoding='utf-8')

In [ ]:
Data.rename(columns={'原始報聘日期': 'str_date'},inplace=True)

# Catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
import catboost

# Random Forest

In [ ]:
'''
==========================================    脫退模型    ==========================================
'''

'''
X=['年度','月份','當月所得','首佣', '續佣', '業績獎金', '其他(總扣F、R、O)',\
    '學位代號','age','本次報聘年資', '年資等級','首佣_組內PR值','續佣_組內PR值', '業績獎金_組內PR值',\
    '其他(總扣F、R、O)_組內PR值', '當月所得_組內PR值', '職級分類_0PSR_到_5DM','當月有增員','當月增員人數(登錄)','性別','身份證開頭_數字']
'''
from sklearn import cross_validation, ensemble, preprocessing, metrics
X=[ 'DoB', 'str_date',  'Time', 'JOURNAL_NO',
    'FYC', 'RYC', 'OR', 'Other', 'Total', 'work_month','age','exp','sex',
   'Wage_YoY','W_Rank_by_Pos','delta_1m','delta_2m','delta_3m',
  'W_Rank_by_exp']
#Data=df
#僅保留"總在職期間_至20171231_月"大於6個月者
bad     =  Data[(Data['Left_3m']==1)]
good    =  Data[(Data['Left_3m']==0)]
print('bad smaple has '+str(len(bad))+' samples.')
print('good smaple has '+str(len(good))+' samples.')

K = 5
kf = KFold(K, True)
bad_sample = resample(bad,replace=True,n_samples=1000)
good_sample = resample(good,replace=True,n_samples=1000)
good_gen = kf.split(bad_sample)
bad_gen = kf.split(good_sample)
for ii in range(K):
    good_train_index, good_test_index = next(good_gen)
    bad_train_index, bad_test_index = next(bad_gen)
    X_train = pd.concat([good_sample.iloc[good_train_index][X],\
                         bad_sample.iloc[bad_train_index][X]])
    X_test = pd.concat([good_sample.iloc[good_test_index][X],\
                        bad_sample.iloc[bad_test_index][X]])
    y_train = pd.concat([good_sample.iloc[good_train_index]['Left_3m'],\
                         bad_sample.iloc[bad_train_index]['Left_3m']])
    y_test = pd.concat([good_sample.iloc[good_test_index]['Left_3m'],\
                        bad_sample.iloc[bad_test_index]['Left_3m']])
    classifier = RandomForestClassifier(n_estimators=300, \
                                        criterion="gini", \
                                        max_features=1, \
                                        n_jobs=-1,\
                                        max_depth=100,
                                        class_weight={0:9,1:1}
                                        #class_weight="balanced"
                                       )
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    #print(confusion_matrix(y_test, y_pred))
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(accuracy)
    #print(classification_report(y_test, y_pred))

importances = classifier.feature_importances_
indices = np.argsort(importances)
cols = [X_train.columns[x] for x in indices]
plt.figure(figsize=(10,6))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center') 
plt.yticks(range(len(indices)), cols) 
plt.xlabel('Relative Importance')

In [ ]:
Data.head()

In [ ]:
Data=df.copy()

In [ ]:
cols

# Decision Tree

In [ ]:
tree = DecisionTreeClassifier(criterion='gini',max_depth=7,random_state=0)

In [ ]:
X=[ '生日', '原始報聘日期',  'Time', 'JOURNAL_NO',
    'FYC', 'RYC', 'OR', 'Other', 'Total', '工作月份','年齡','年資','性別' ]

#僅保留"總在職期間_至20171231_月"大於6個月者
bad     =  Data[(Data['Left_3m']==1)]
good    =  Data[(Data['Left_3m']==0)]
print('bad smaple has '+str(len(bad))+' samples.')
print('good smaple has '+str(len(good))+' samples.')


K = 5
kf = KFold(K, True)
bad_sample = resample(bad,replace=True,n_samples=1000)
good_sample = resample(good,replace=True,n_samples=1000)
good_gen = kf.split(bad_sample)
bad_gen = kf.split(good_sample)
for ii in range(K):
    good_train_index, good_test_index = next(good_gen)
    bad_train_index, bad_test_index = next(bad_gen)
    X_train = pd.concat([good_sample.iloc[good_train_index][X],\
                         bad_sample.iloc[bad_train_index][X]])
    X_test = pd.concat([good_sample.iloc[good_test_index][X],\
                        bad_sample.iloc[bad_test_index][X]])
    y_train = pd.concat([good_sample.iloc[good_train_index]['Left_3m'],\
                         bad_sample.iloc[bad_train_index]['Left_3m']])
    y_test = pd.concat([good_sample.iloc[good_test_index]['Left_3m'],\
                        bad_sample.iloc[bad_test_index]['Left_3m']])
    classifier = DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    #print(confusion_matrix(y_test, y_pred))
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(accuracy)
    #print(classification_report(y_test, y_pred))
from sklearn.tree import export_graphviz
export_graphviz(classifier)

In [ ]:
import graphviz
export_graphviz(classifier,feature_names=X)
graph = graphviz.Source('tree.dot')

# Below are Useless

In [ ]:
Data.shape

In [ ]:
Data.groupby(by='AGNTNUM')

In [ ]:
Data[Data['AGNTNUM']==10223]

In [ ]:
income_data.shape

In [ ]:
young_agent=temp.copy()

In [ ]:
temp = aa_data.copy()
#計算年齡、年資
temp['Age']=(20190726-temp['生日'])/10000
temp['Seniority']=(20190726-temp['原始報聘日期'])/10000
#取年齡年資Dif>20，確保資料無誤
temp['Dif']=temp['Age']-temp['Seniority']
temp=temp[(temp['Dif']>=20)&(temp['Age']<=70)]
#篩出仍在職的人員
temp= temp[(temp['解聘日期 ']==99999999)]
temp.shape
#篩出職級為E、C的人員
temp['職務代號']=np.array(temp['職務代號'].tolist()).astype('<U1')
temp1=temp[temp['職務代號'].isin(['E','D'])]
temp2=temp[temp['職務代號'].isin(['C','E','D'])]
#10779
#10769

In [ ]:
print(temp1.shape,temp2.shape)

In [ ]:
#DE
data=temp1
from scipy.stats import gaussian_kde
x=data['Age']
y=data['Seniority']
# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)
data.plot.scatter(x='Age',y='Seniority',s=45,c=z, edgecolor='')
#len(data[data['Age']<=30])/len(data)
#行政院 網路重度使用者 年稜分布

In [ ]:
len(temp1[temp1['Age']<=37])/len(temp1)

In [ ]:
#CDE
data=temp2
from scipy.stats import gaussian_kde
x=data['Age']
y=data['Seniority']
# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)
#data.plot.scatter(x='Age',y='Seniority',s=45,c=z, edgecolor='')

In [ ]:
len(temp2[temp2['Age']<=37])/len(temp2)

In [ ]:
temp

In [ ]:
temp.plot.scatter(x='Age',y='Seniority',s=45,c=z, edgecolor='')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x,y, c=z, s=25, edgecolor='')
plt.show()

In [ ]:
Data.to_csv('Resign_Rate_Model.csv')